In [1]:
import ee
import os
import pandas as pd
import seaborn as sns
import geopandas as gpd
import geemap
import rasterio as rio
import pprint
import time
import gc
import numpy as np

from sklearn.preprocessing import Normalizer
from numpy import unique
from rasterio import features
from osgeo import gdal
from osgeo import osr
from datetime import datetime
from dateutil import relativedelta
from datetime import timedelta


In [2]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [3]:
x = "-55.3800349723379242,-8.0051619964282423,-54.7129565837871183,-7.5177835013920964"
x = x.split(',')
x = [[float(x[i]), float(x[i+1])] for i in range(0,len(x),2)]


geometria = geometry = ee.Geometry.Polygon(x)

In [8]:
def maskEmptyPixels(image): 
  withObs = image.select('num_observations_1km').gt(0)
  return image.updateMask(withObs)


def maskClouds(image): 
  QA = image.select('state_1km')
  bitMask = 1 << 10
  return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))

collection = ee.ImageCollection('MODIS/006/MOD09GA')\
        .filterDate('2010-04-01', '2010-05-01')\
        .map(maskEmptyPixels)

totalObsCount = collection\
        .select('num_observations_1km')\
        .count()

collectionCloudMasked = collection.map(maskClouds)

clearObsCount = collectionCloudMasked\
        .select('num_observations_1km')\
        .count()\
        .unmask(0)
        

# Create a default map
Map = geemap.Map()
Map.addLayer(
    collectionCloudMasked.median(),
    {'bands': ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
     'gain': 0.07,
     'gamma': 1.4
    },
    'median of masked collection'
  )
Map.addLayer(
    totalObsCount,
    {'min': 84, 'max': 92},
    'count of total observations',
    False
  )
Map.addLayer(
    clearObsCount,
    {'min': 0, 'max': 90},
    'count of clear observations',
    False
  )
Map.addLayer(
    clearObsCount.toFloat().divide(totalObsCount),
    {'min': 0, 'max': 1},
    'ratio of clear to total observations'
  )
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [31]:

# Dataset 
image = ee.ImageCollection("MODIS/006/MOD13Q1")\
        .filterDate('2000-02-18','2005-01-01')\
        .median()\
        .clip(geometria)

bands = ['EVI', 'NDVI']

image = image.select(bands)
# Select all bands and NDVI for the classification
#print(image.bandNames().getInfo())


# Perform K-means clustering
training_dataset = image.sample(
    **{
        'scale': 250,
        'numPixels': 500,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)


clusters_kmeans = ee.Clusterer.wekaKMeans(9).train(training_dataset)

# Classify the image
classified = image.cluster(clusters_kmeans)

# Create a mask for each of the 8 clusters
mask0 = classified.eq(0)
mask1 = classified.eq(1)
mask2 = classified.eq(2)
mask3 = classified.eq(3)
mask4 = classified.eq(4)
mask5 = classified.eq(5)
mask6 = classified.eq(6)
mask7 = classified.eq(7)
mask8 = classified.eq(8)



masks = [mask0, mask1, mask2, mask3, mask4, mask5, mask6, mask7, mask8]


In [30]:
# Create a default map
Map = geemap.Map()
Map.addLayer(classified.randomVisualizer(), {} , 'teste')
Map.addLayer(masks[0], {'palette': ['white', 'black']}, 'Cluster '+ str(0))
Map.addLayer(masks[1], {'palette': ['white', 'black']}, 'Cluster '+ str(1))
Map.addLayer(masks[2], {'palette': ['white', 'black']}, 'Cluster '+ str(2))
Map.addLayer(masks[3], {'palette': ['white', 'black']}, 'Cluster '+ str(3))
Map.addLayer(masks[4], {'palette': ['white', 'black']}, 'Cluster '+ str(4))
Map.addLayer(masks[5], {'palette': ['white', 'black']}, 'Cluster '+ str(5))
Map.addLayer(masks[6], {'palette': ['white', 'black']}, 'Cluster '+ str(6))
Map.addLayer(masks[7], {'palette': ['white', 'black']}, 'Cluster '+ str(7))
#Map.addLayer(mask0.randomVisualizer(), {} , 'mask')

# Display the map
Map

EEException: GeometryConstructors.Polygon: LinearRing requires at least 3 points.